# Flask App for Olympics Dashboard

#### By Safaa Amer, Maya Dennis, Christa Fields, and Kevin Yao

In [257]:
# import dependecies
from flask import Flask, jsonify, render_template
# note: importing various sqlalchemy methods for reading in sqlite database 
from sqlalchemy import create_engine, inspect, MetaData
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

In [258]:
app = Flask(__name__)



In [259]:
# sqlalchemy 'create_engine' connect to sqlite database 
engine = create_engine("sqlite:///data/newsAPI.sqlite")
#    automapping the columns to sqlalchemy base object
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [260]:
# save list of column names to var
inspector = inspect(engine)

inspector.get_table_names()

['Google_trends_results', 'Guard_api_results', 'NYT_api_results']

In [261]:
# saving references to each table
NYT_results = Base.classes.NYT_api_results
Guard_results = Base.classes.Guard_api_results
Google_results = Base.classes.Google_trends_results
print(NYT_results)

<class 'sqlalchemy.ext.automap.NYT_api_results'>


In [262]:
# creating session
session = Session(engine)

In [263]:
# Flask route: index
@app.route('/')
def index():
    return render_template("index.html")

In [264]:
# Flask route: NYT api 
@app.route('/nyt_articles')
def NYT_articles():
    NYT_ls = []
    query = session.query(NYT_results.news_id, NYT_results.year,\
                     NYT_results.article_hits).all()
    for instance in query:
        NYT_ls.append({"news_id":instance[0], "year":instance[1],\
                       "article_hits":instance[2]})
    return jsonify(NYT_ls)

In [265]:
# Flask route: theGuardian api
@app.route("/guardian_articles")
def Guard_articles():
    Guard_ls = []
    query = session.query(Guard_results.news_id, Guard_results.year,\
                     Guard_results.article_hits).all()
    for instance in query:
        Guard_ls.append({"news_id":instance[0], "year":instance[1],\
                       "article_hits":instance[2]})
    return jsonify(Guard_ls)
    

In [ ]:
# Flask route: Google trends
@app.route("/google_trends")
def Google_trends():
    Goog_ls = []
    query = session.query(Google_results.month,\
                     Google_results.trend).all()
    for instance in query:
        Goog_ls.append({"month":instance[0],\
                       "trend":instance[1]})
    return jsonify(Goog_ls)
    

In [ ]:
# Run app 
if __name__ == "__main__":
    app.run(debug=False)
    # if there's any problems, set debug =False

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2018-03-14 21:29:32,359] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\K Y\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\K Y\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\K Y\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\K Y\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\K Y\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\K Y\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-263-a33b5b4d5a28>",